#  Construction de la base de données

A partir des données disponible sur le site [Tennis Temple](https://fr.tennistemple.com/), nous allons construire une base de données compilant les statistiques des matchs joués les dix dernières années sur le circuit ATP.

### Importation des modules

In [ ]:
import urllib
import bs4
import selenium
import collections
import pandas as pd
import numpy as np

### Fonction liminaire

La fonction get_page(url) permet de récupérer l'ensemble du code source de la page web identifiée par l'extension url en argument sur le site [Tennis Temple](https://fr.tennistemple.com/)

In [ ]:
from unidecode import unidecode

def get_page(url):
    
    url_tournoi = 'https://fr.tennistemple.com' + url
    url_tournoi = unidecode(url_tournoi) #Permet de retirer les accents de l'url
    req = urllib.request.Request(url_tournoi)
    html = urllib.request.urlopen(req).read()
    
    return bs4.BeautifulSoup(html, "lxml")

### Récupération de tous les tournois joués les 10 dernières années

Sur la page [Tennis temple calendrier](https://fr.tennistemple.com/calendrier-atp/) sont référencés par mois et par année les différentes éditions des tournois inscrits au calendrier ATP.
Nous cherchons donc à récupérer les noms et les extensions url de ces tounois.

In [1]:
# Vérifier qu'il n'y a pas de bug pour décembre où certaines années aucun tournoi n'est joué

liste_mois = [(1,janvier),(2,fevrier),(3,mars),(4,avril),(5,mai),(6,juin),(7,juillet),(8,aout),(9,septembre),(10,octobre),(11,novembre),(12,decembre)]

for a in range (2006,2019):
    
    for m in liste_mois :
        
        url_tournoi = '/calendrier-atp/' + m[1] + "-"+ m[0] + "-" + a
        page = get_page(url_tournoi)
        
        Liste_url_tournoi = []
        Liste_tournoi = []

        for item in page.findAll('div', {'itemtype' : 'http://schema.org/SportsEvent'}): 
            
            for tournoi in item.findAll({'a'}) :
                
                Liste_url_tournoi.append(tournoi.get("href"))
                
                # Nettoyage du nom du tournoi
                nom_tournoi = tournoi.getText()
                nom_tournoi=nom_tournoi.lstrip('\n').lstrip('\t')
                nom_tournoi=nom_tournoi.rstrip('\n').rstrip('\t').rstrip('\n')
                
                Liste_tournoi.append(nom_tournoi)

SyntaxError: invalid syntax (<ipython-input-1-374c52846bf6>, line 7)

### Données sur les tournois

In [ ]:
# Regarder si pas plus élégant avec un tableau numpy

Liste_nom = []
Liste_location = []
Liste_categorie = []
Liste_surface = []
Liste_date = []
Liste_dotation = []

for url in Liste_url_tournoi:
    
    page_tournoi = get_page(url)
    
    for item in page_tournoi.findAll('div', {'class' : 'ts_d_3'}):
        intitule = item.find('div', {'class' : 'ts_d_4'}).getText()
        
        if intitule == "Nom officiel" :
            Liste_nom.append(item.find('div', {'class' : 'ts_d_5'}).getText())
        elif intitule == "Ville" :
            Liste_location.append(item.find('div', {'class' : 'ts_d_5'}).getText())
        elif intitule == "Catégorie" :
            Liste_categorie.append(item.find('div', {'class' : 'ts_d_5'}).getText())
        elif intitule == "Surface" :
            Liste_surface.append(item.find('div', {'class' : 'ts_d_5'}).getText())
        elif intitule == "Date" :
            Liste_date.append(item.find('div', {'class' : 'ts_d_5'}).getText())
        elif intitule == "Dotations" :
            Liste_dotation.append(item.find('div', {'class' : 'ts_d_5'}).getText())

df_tournoi = pd.DataFrame({'Nom' : Liste_tournoi, 'Nom officiel' : Liste_nom, 'Ville' : Liste_location, 'Categorie' : Liste_categorie, 'Surface' : Liste_surface, 'Date' : Liste_date, 'Dotations' : Liste_dotation, 'Url' : Liste_url})

In [ ]:
df_tournoi.head()

### Récupération des matchs joués dans chacun des tournois

A partir de la page de chacun des tournois, il est possible d'accéder au tableau de ce-dernier, référençant l'ensemble des matchs qui ont été joués.

In [ ]:
Liste_url_match = []

for url in Liste_url_tournoi:
    
    page_tableau = get_page(url + '/tableau')
    
    for item in page_tableau.findAll('a',{'style' : 'padding-left:2px;color:#515151;font-size:11px;font-style:italic;'}):
    
        Liste_url_match.append(item.get("href"))

### Informations sur les matchs joués

In [ ]:
tab = numpy.zeros(len(Liste_url_match),23)

for i in range(len(Liste_url_match)) :
    
    url = Liste_url_match[i]
    
    page_match = get_page(url)
    
    # Liste des noms
    Liste_Nom = page_match.findAll('span', {'itemprop' : 'name'})
    
    #Nom tournoi
    tab[i,0] = Liste_Nom[0].getText()

    #Nom du premier joueur
    tab[i,1] = Liste_Nom[1].find('b').getText()
    
    #Prenom du premier joueur
    tab[i,2] = Liste_Nom[1].find('span', {'class' : 'first_name'}).getText()[2:]
    
    #Nom du deuxieme joueur
    tab[i,3] = Liste_Nom[2].find('b').getText()
    
    #Prenom du deuxieme joueur
    tab[i,4] = Liste_Nom[2].find('span', {'class' : 'first_name'}).getText()[2:]
    
    #Avancement dans le tournoi
    
    # tab[i,5]
    # Regarder quelque chose avec href = l'url du tableau du tournoi
    
    #Duree du match
    tab[i,6] = page_match.find('div', {'class' : 'status status_2'}).getText()[16:]
    
    #Temperature
    tab[i,7] = page_match.find('div', {'style' : 'float:right;margin-right:8px;color:#c9c9c9;font-size:11px;'}).getText()
    
    #Score
    #1er set
    tab[i,8] = page_match.find('div', {'class' : 'p1'}).find('div', {'class' : 'set1'}).getText()
    tab[i,9] = page_match.find('div', {'class' : 'p2'}).find('div', {'class' : 'set1'}).getText()
    
    #2eme set
    tab[i,10] = page_match.find('div', {'class' : 'p1'}).find('div', {'class' : 'set2'}).getText()
    tab[i,11] = page_match.find('div', {'class' : 'p2'}).find('div', {'class' : 'set2'}).getText()
    
    #3eme set
    tab[i,12] = page_match.find('div', {'class' : 'p1'}).find('div', {'class' : 'set3'}).getText()
    tab[i,13] = page_match.find('div', {'class' : 'p2'}).find('div', {'class' : 'set3'}).getText()
    
    #4eme set
    tab[i,14] = page_match.find('div', {'class' : 'p1'}).find('div', {'class' : 'set4'}).getText()
    tab[i,15] = page_match.find('div', {'class' : 'p2'}).find('div', {'class' : 'set4'}).getText()
    
    #5eme set
    tab[i,16] = page_match.find('div', {'class' : 'p1'}).find('div', {'class' : 'set5'}).getText()
    tab[i,17] = page_match.find('div', {'class' : 'p2'}).find('div', {'class' : 'set5'}).getText()
    
    # faire des conditions sur le non vide, pour le cas des matchs forfaits
    
    #Stats match
    if page_match.find('div', {'class' : 'set set0'}) :
        tab_match = page_match.find('div', {'class' : 'set set0'})
        tab[i,17] = tab_match.find('div', {'class' :'tt_service_game_won_rap'}).find('div', {'class' : 'p1'}).getText()
    #Stats 1er set
    
    #Stats 2eme set
    
    #Stats 3eme set
    
    #Stats 4eme set
    
    #Stats 5eme set
    